In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests2

In [2]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    x = np.random.uniform(low=-3., high=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs

yn,xn,nobs = gen_data()

In [3]:
def compute_loglike(resid):
    sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
    ll = np.log(stats.norm.pdf(resid,loc=0,scale=sigma))
    return ll

In [4]:
def compute_score(yn,xn,params):
    
    xn = sm.add_constant(xn)
    resid = yn - np.matmul(xn,params)
    
    k = len(params)
    scale = (resid**2).mean()
    
    tile_resid = np.tile( resid, k)
    tile_resid = np.reshape(tile_resid, (k,xn.shape[0]) ).transpose()
    grad = tile_resid*xn/scale     
    return grad
    
def compute_hess(yn,xn,params):
    pass


def setup_shi(yn,xn):
    x1n,x2n = xn[:,0:2],xn[:,1:3]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    k1 = len(model1_fit.params)
    
    ll1 = compute_loglike(model1_fit.resid)
    grad1 =  compute_score(yn,x1n,model1_fit.params)
    hess1 = model1.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    k2 = len(model1_fit.params)
    
    ll2 = compute_loglike(model2_fit.resid)
    grad2 =  compute_score(yn,x2n,model2_fit.params)
    hess2 = model2.hessian(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [5]:
reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200,use_boot2=False)
print("reg: %s, boot: %s, boot_nml: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2, llr, std, omega ))

reg: [1. 0. 0.], boot: [1. 0. 0.], boot_nml: [0.95 0.02 0.03], llr:-0.028442326631734467, std: 0.7557582943812067, omega:0.03770790070264348


In [6]:
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

[1. 0. 0.]


In [7]:
sys.path.append("../")
import vuong_tests2

#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 1*x[:,2] + 1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs

reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.99 0.   0.01], boot1: [0.99 0.   0.01], boot2: [0.99 0.   0.01], llr:1.9947122924982827, std: 26.328373346183167, omega:0.9939587491731376
[0.97 0.02 0.01]


In [7]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + 2*x[:,2] + 2*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.947 0.028 0.025], boot1: [0.941 0.029 0.03 ], boot2: [0.946 0.027 0.027], llr:-0.4469315995409912, std: 31.62428160591081, omega:0.998783658368862
[0.97 0.02 0.01]


In [9]:
#generate the sample
def gen_data(beta=1):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(1000,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.998 0.    0.002], boot2: [0.926 0.035 0.039], llr:-0.01566332351808678, std: 0.9765522484063855, omega:0.03957152829705613
[1. 0. 0.]


In [10]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .1*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:42.14443836265048, std: 9.360281640205567, omega:0.28459879497573415
[0. 1. 0.]


In [11]:
#generate the sample
def gen_data(beta=3):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.99 0.01 0.  ], boot1: [0.98 0.02 0.  ], boot2: [0.85 0.13 0.02], llr:0.4686314163166089, std: 1.7677629442230747, omega:0.04476886754116609
[1. 0. 0.]


In [12]:
#generate the sample
def gen_data(beta=2):
    nobs = 1000
    #x = np.random.normal(low=-3., high=3., size=(nobs,3))
    x = np.random.normal(scale=3., size=(nobs,3))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + .01*x[:,0] + beta * x[:,1] + e
    return y,x,nobs


reg,boot1,boot2, llr, std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi,trials=200)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))
shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [1. 0. 0.], boot1: [0.99 0.01 0.  ], boot2: [0.84 0.14 0.02], llr:0.5386244554740763, std: 1.2225196049908784, omega:0.04425787532172899
[1. 0. 0.]
